In [1]:
from npodjlEnv import NpodJLEnv
import numpy as np
env = NpodJLEnv()
env.ver()

0.3

In [2]:
def argmaxQ(Q, s, n_actions):
    if s in Q.keys():
        return(np.argmax(Q[s]))
    else:
        return(np.random.randint(0, n_actions))

In [3]:
def policy_greedy(Q, s, epsilon, n_actions):
    if np.random.rand() < epsilon:
        return(argmaxQ(Q,s,n_actions))
    else:
        return(np.random.randint(0, n_actions))

In [4]:
def SARSA(env, alpha, gamma, epsilon, Q, max_cylces):
    n_actions = len(env.actions())
    r = [None] * (2*max_cycles)
    s = [None] * (2*(max_cycles+1))
    a = [None] * (2*max_cycles)
    
    t=0
    s[t] = env.encoded_state()
    a[t] = policy_greedy(Q,s[t], epsilon, n_actions)
    obs = env.run(a[t])
    r[t] = obs['reward']
    s[t+1] = env.encoded_state() 
    if not s[t] in Q.keys():
        Q[s[t]] = [0]*n_actions
    try:
        while t < max_cycles:
            a[t+1] = policy_greedy(Q,s[t+1], epsilon, n_actions)
            obs = env.run(a[t])
            r[t+1] = obs['reward']
            s[t+2] = env.encoded_state()
            if not s[t+1] in Q.keys():
                Q[s[t+1]] = [0]*n_actions
            Q[s[t]][a[t]] = Q[s[t]][a[t]] + alpha * ( r[t] + (gamma * Q[s[t+1]][a[t+1]] ) - Q[s[t]][a[t]] )
            t += 1
            
    except KeyboardInterrupt:
        print("Press Ctrl-C to terminate while statement")
        pass
    return(s,a,r,Q)

In [13]:
Q = dict()
env.reset()

In [ ]:
alpha = 1
gamma = 0.99
epsilon = 0.5
max_cycles = 500
sol = SARSA(env, alpha, gamma, epsilon, Q, max_cycles)

In [7]:
Q

{'cycles': 14,
 'theta': array([[4.38162078e-02, 9.85168451e-01, 1.05556749e-01, 1.66226307e-02,
         6.26544119e-02, 9.53421887e-02, 7.14400410e-02, 1.17470775e-01,
         2.90526231e-01, 1.03795604e-01, 3.31855444e-01, 6.26898722e-02,
         3.41158837e-01, 6.85837748e-02, 3.08501185e-01, 1.85042942e-02,
         3.49182465e-01, 3.21667103e-01, 3.32218496e-01, 2.81664237e-01,
         2.84787535e-01, 2.82302431e-01, 3.12792066e-01, 9.22709202e-02,
         3.52920192e-02, 3.01641969e-01, 3.54086005e-01, 4.62939301e-02,
         2.92004025e-01, 8.74856888e-02, 9.83784296e-02, 3.12671819e-01,
         8.19732570e-02, 2.90616194e-01, 3.54421722e-01, 9.14705053e-02,
         5.60928691e-02, 8.68176517e-02, 1.30653974e-01, 7.64657307e-02,
         8.70593191e-02, 2.90550698e-01, 5.58308436e-02, 2.93281811e-01,
         1.30894551e-01],
        [7.59888761e+01, 2.12792084e+02, 1.39545785e+02, 1.15993209e+02,
         9.09419463e+01, 7.00813183e+01, 6.51682847e+01, 7.80166137e+01,
 